In [1]:
from pytube import YouTube
import pandas as pd
import numpy as np
from xml.etree import ElementTree as ET
import bleach
import re
import pattern3


import gensim
from gensim import corpora, models, similarities, matutils
from gensim.summarization import keywords
from gensim.summarization.summarizer import summarize

# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)





In [2]:
def get_transcript(url):
    path=url
    yt = YouTube(path)
    caption = yt.captions.get_by_language_code('en')
    xml=caption.xml_captions
    root = ET.fromstring(xml)
    #gets the transcripts
    doc=''
    for child in root:
        doc=doc+" "+(child.text)
    return doc.replace('\n',' ')

In [114]:
def make_corpus(url_list):
    corpus=[]
    for url in url_list:
        x=bleach.clean(get_transcript(url), tags=[], attributes={}, styles=[], strip=True)
        y=re.sub(r'&#39;', '', x)
        z=re.sub(r'\[inaudible]', '', y)
        doc=re.sub(r'\[Music]', '', z)
       
        corpus.append(doc)
    
    return corpus

In [4]:
def get_topic_space(url_list):
    docs=make_corpus(url_list)
    vectors_list=[]
    for i in range(len(docs)):
        keys=keywords(docs[i], words=5,lemmatize='True').split('\n')
        vector=sum(list(map(model.word_vec,keys)))
        vectors_list.append(vector)
   
    return (sum(vectors_list)/len(docs))
                     
 

In [5]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors

glove_file = datapath('/Users/andrewportal/Downloads/glove/glove.6B.50d.txt')
tmp_file = get_tmpfile("glove_word2vec.txt")

# call glove2word2vec script
# default way (through CLI): python -m gensim.scripts.glove2word2vec --input <glove_file> --output <w2v_file>
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_file, tmp_file)

model = KeyedVectors.load_word2vec_format(tmp_file)    

In [80]:
#3blueonebrown, gameranx review of metro exdus, vice news yemen
url_list=('https://www.youtube.com/watch?v=BbHLPBJvSOc&t=2701s'
        'https://www.youtube.com/watch?v=fdaVySF_-FQ&feature=youtu.be',
        'https://www.youtube.com/watch?v=RWOPlynTcmk')

In [115]:
data=make_corpus(url_list)

In [84]:
d='Watching the daily destruction of our civility to one another., Putting up silly things like this all right, so it really unbecoming of Congress. Its that sort of behavior [, inaudible, ], That., Im responsible for Im responsible for your silliness, because I did the same thing that youre Doing now, 410- year. - I protected Mr trump for ten years. And the fact that you pull up a news article that has no value to it and you want to use that is the premise for discrediting me. That Im, not the person that people cold at three Oclock in the morning would make you inaccurate in actuality, would make you a liar. Which puts you into the same position that I am in., And I can only warn people. The more people that follow. Mr trump, as I did, blindly., Are going to suffer the same consequences that Im suffering.. What warning would you give young people Who are tempted, as you were, Would you encourage them not to wait ten years to see the light., But the advice? Would you give young people in particular young lawyers., So they do not abuse their bar license? As you did. To whats happened to me., I had a wonderful life. I have a beautiful wife. I have two amazing children. Hi chief financial success by the age of thirty. Nine, I didnt go to work for Mr trump. Because I had to. I want to work for him because I wanted. To and Ive lost it all.. So if Im, not a picture, perfect thats the picture that should be up there. If Im not a picture. Perfect example of what not to do. Thats the example that Im trying to set for my children., You make mistakes in life and Ive, owned them, and Ive taken responsibility for them. And Im paying a huge price, as is my family, so fat in and of itself. Isnt enough to dissuade somebody from acting in the callous manner that I did. Im, not sure that.. That person has any -- Any chance very much like how I mean right, now., A recurring theme in your testimony is concerned for your family safety.. What, specifically, are you most concerned about For the president.? Unlike my calling for trump that has a thousand followers, hes got over sixty million people. And when Mr trump turned around early in the campaign and said I can shoot somebody on Fifth Avenue and get away with it.. I want to be very clear. Hes, not sure the truth is you dont know him. I do. Ive sat next to this man for ten years and I watched his bac. Im, the one who started the campaign. And Im the one who continued in twenty fifteen to promote them.. So many things I thought that he can do. That is just great and he can and he is doing things that are great., But this destruction of our civility to one another is just its out of control and when he goes on Twitter and he starts bringing in my in laws my parents, my wife, which He think is gon na happen. Hes, causing hes, sending out the same message that he can do whatever he wants, misses his country coming in order cracked. And hopefully something bad will happen to me or my children and my wife. So that I will not be hereand, Testify thats what his hope was was to intimidate me and again I sent to everybody. Who joined and said that this is just not right. Have you ever seen, Mr trump personally threaten people with the physical harm, [, inaudible, ] No. One he would use others.. He would hire other people to do that, Im, not so sure that it gets harder than they already working. There. Everybodys job at the trump organization is to protect Mr trump. Every day, most of us and we were coming in and we were going to lie for him on some things. And that became the norm and thats exactly whats happening right now in in this country, And its exactly whats happening here in governments are.. Thank you, Mr chairman. My time is expired. John from. Thank you.. Mister Adams, the chairman, can, can we take a break [, inaudible, ] Right now, [, inaudible, ], All right, [, inaudible ], I thats. Okay. Thank you.'

In [116]:
data[0]

' . Watching the daily destruction of our civility to one another. Putting up silly things like this all right so it really unbecoming of Congress. Its that sort of behavior  That. Im responsible for Im responsible for your silliness because I did the same thing that youre doing now 410- year. I protected Mr trump for ten years. And the fact that you pull up a news article that has no value to it and you want to use that is the premise for discrediting me. That Im not the person that people cold at three oclock in the morning would make you inaccurate in actuality would make you a liar. Which puts you into the same position that I am in. And I can only warn people the more people that follow Mr trump as I did blindly. Are going to suffer the same consequences that Im suffering. What warning would you give young people? Who are tempted as you were? Would you encourage them not to wait ten years to see the light. But the advice would you give young people in particular young lawyers. So 

In [112]:
keywords(d, words=10,lemmatize='True', split='True')

['people',
 'thing',
 'years',
 'trump',
 'children',
 'wife',
 'right',
 'hes',
 'perfect',
 'bar']

In [35]:
from gensim.summarization.textcleaner import HAS_PATTERN
from gensim.utils import has_pattern

assert HAS_PATTERN
assert has_pattern

In [36]:
from gensim.parsing.preprocessing import remove_stopwords

In [38]:
x=remove_stopwords(data[1])

In [92]:
keywords(d,words=10,pos_filter=('NN','NNS','JJ','NNP','NNPS'),lemmatize=True,scores=True)

[('people', 0.22915405136007364),
 ('thing', 0.21597442963367175),
 ('years', 0.18755444096725907),
 ('trump', 0.17429386183583906),
 ('children', 0.1602520386167039),
 ('wife', 0.15681738354091948),
 ('right', 0.15043728741443826),
 ('hes', 0.1420620362528735),
 ('adams', 0.13921578863566716),
 ('perfect', 0.13921578863566708)]

In [110]:
keywords(d, words=10,pos_filter=('NN','NNS','NNPS','NNP'), lemmatize='True',split=True)

['trump',
 'family',
 'bar',
 'article',
 'civility',
 'premise',
 'thing',
 'silliness',
 'children',
 'parents']

In [91]:
summarize(d, ratio=.1)

'Watching daily destruction civility another., Putting silly things like right, unbecoming Congress.\nThat person -- Any chance like I mean right, now., A recurring theme testimony concerned family safety..\nUnlike calling trump thousand followers, hes got million people.\nHave seen, Mr trump personally threaten people physical harm, [, inaudible, ] No. One use others..\nHe hire people that, Im, sure gets harder working.'

In [33]:

ttext_model=KeyedVectors.load_word2vec_format('Users/andrewportal/Downloads/glove/fast_wiki.vec')

FileNotFoundError: [Errno 2] No such file or directory: 'Users/andrewportal/Downloads/glove/fast_wiki.vec'

In [25]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors

glove_file = datapath('/Users/andrewportal/Downloads/glove/glove.6B.50d.txt')
tmp_file = get_tmpfile("glove_word2vec.txt")

# call glove2word2vec script
# default way (through CLI): python -m gensim.scripts.glove2word2vec --input <glove_file> --output <w2v_file>
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_file, tmp_file)

model = KeyedVectors.load_word2vec_format(tmp_file)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/andrewportal/Downloads/glove/glove.6B.50d.tt'

In [34]:
model.n_similarity(['obama', 'president'], ['clinton', 'president'])

0.9893647516162923

In [233]:
x=keywords(data[0], words=5,lemmatize='True').split('\n')

In [234]:
y=keywords(data[1], words=5,lemmatize='True').split('\n')

In [235]:
z=keywords(data[2], words=5,lemmatize='True').split('\n')

In [236]:
vector1=list(map(model.word_vec,x))
vector2=list(map(model.word_vec,y))
vector3=list(map(model.word_vec,z))

In [237]:
test1=(sum(vector1))
test2=(sum(vector2))
test3=(sum(vector3))


In [247]:
test_list=(test1,test2,test3)

In [248]:
vecl=sum(test_list)/len(data)

In [249]:
vecl

array([ 7.4131131e-01, -7.3579568e-01,  1.0339093e+00,  8.2039875e-01,
        5.3618300e-01,  6.2923334e-03, -5.4832882e-01,  2.6783338e-02,
       -4.4474664e-01,  1.5703840e+00, -1.6001169e-01,  5.9797329e-01,
       -2.6063004e+00,  1.9161171e+00, -7.6865354e-03,  5.4009300e-01,
        1.5315199e+00,  1.1847132e+00, -1.2932111e+00, -2.0422976e+00,
       -6.0389662e-01, -9.2646670e-01,  5.3122962e-01,  8.8260442e-01,
        6.4226031e-01, -4.2216668e+00, -1.5525804e+00,  6.0834354e-01,
        2.5877457e+00, -1.8794533e+00,  9.7607508e+00,  2.4767368e+00,
       -8.1350070e-01, -7.9928875e-01, -1.1277546e+00, -5.8675331e-01,
       -4.2665729e-01, -3.0934092e-01,  4.7505701e-01, -1.3624351e-01,
       -1.8324178e+00, -8.3093101e-01, -1.7135878e+00,  1.2964234e+00,
        4.0663481e-03,  1.1034474e+00,  1.9586933e-01, -2.7896002e-01,
        7.9566002e-02,  1.3572668e+00], dtype=float32)

In [55]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(a,b):
    return dot(a, b)/(norm(a)*norm(b))

In [241]:
cos_sim(vecl,check)

1.0

In [216]:
check=get_topic_space(url_list)

In [16]:
model[oov_word]

NameError: name 'model' is not defined

In [13]:
import spacy
spacy.load('en_core_web_sm')
from spacy.lang.en import English, STOP_WORDS

    

In [14]:
sen_new =([word for word in data[0] if word not in STOP_WORDS])

In [16]:
np.load('wall.npy')

array([ 2.9732408e-02,  1.0274980e+00,  2.3309181e+00, -1.2149209e+00,
       -5.6307471e-01,  8.6201817e-01, -1.3634742e+00,  1.3680534e-01,
        6.2956548e-01, -5.5167639e-01,  3.3749912e-03, -2.8777892e-02,
        7.6371378e-01,  2.3457743e-01, -2.8795487e-01, -1.1057765e+00,
        3.1816101e-01,  6.1283487e-01, -2.3613040e+00,  6.6591495e-01,
        1.0877289e+00,  7.9955697e-01,  6.6461629e-01, -1.2556249e+00,
       -3.0047616e-01, -1.5620540e-01, -1.2492443e-01, -2.2859457e+00,
        4.1647029e-01, -1.4113430e+00,  7.8564340e-01,  1.9439722e+00,
        5.6526548e-01,  4.3577844e-01,  5.2492028e-01,  7.0656878e-01,
       -2.0587547e-02,  5.2292788e-01,  1.1313336e+00, -2.7906993e-01,
       -1.6619072e+00, -5.1636994e-01,  8.0358654e-01, -1.1464814e+00,
       -1.4269688e+00, -4.5288235e-01,  8.5908189e-02, -7.4340820e-01,
       -4.9966222e-01, -1.7855331e+00, -1.4613866e-02, -1.1299710e-01,
        4.9723011e-01,  2.8480790e+00, -9.0240228e-01, -6.5354414e+00,
      